In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
from emv.db.dao import DataAccessObject
from emv.db.queries import get_features_by_type_paginated, count_features_by_type
from sqlalchemy.sql import text
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import numpy as np
from emv.api.models import Feature
from emv.api.models import Projection, MapProjectionFeatureCreate
from emv.db.queries import create_projection, create_map_projection_feature, create_feature
from umap import UMAP
import numba

#from emv.features.pose import load_poses 
from emv.client.get_content import get_features
from emv.utils import dataframe_from_hdf5
from emv.settings import DRIVE_PATH

In [ ]:
count_features_by_type("transcript+ner", long_clips_only=True)

In [24]:
MAX_FEATURES = 100000 #count_features_by_type("transcript+ner", short_clips_only=True) + 1
data = get_features_by_type_paginated("transcript+ner", page_size=1000, short_clips_only=True)

while len(data) < MAX_FEATURES:
    last_seen_id = data[-1].get("feature_id", None)
    if last_seen_id is None:
        break
    data.extend(get_features_by_type_paginated("transcript+ner", page_size=1000, last_seen_feature_id=last_seen_id, short_clips_only=True))

df = pd.DataFrame(data)
print(f"Retrieved {len(df)} instances")

Retrieved 100000 instances


In [25]:
df["locations"] = df["data"].apply(lambda x: [w[0] for w in x.get("entities", []) if w[1] == "LOC"])
locations = list(set(df["locations"].explode().values))
print(f"Found {len(locations)} unique locations")

Found 77705 unique locations
